In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
print("Importing spreadsheet...")
CAM_DF = pd.read_excel('../docs/düzenli veriler delphi için.xlsx')
print("Import complete!")

Importing spreadsheet...
Import complete!


In [83]:
def convert_num_base(colStart, colEnd, conv_in, targetDF = CAM_DF):

    # if (colStart not in range(0,10)):
    #     print("Start column input is invalid!")
    #     return -1
    # if (colEnd not in range(0, 10)):
    #     print("End column input is invalid!")
    #     return -1

        
    if conv_in not in ('2', '10', 'ascii'):
        print("Invalid conversion input!")
        return -1

    CAM_DF_conv = targetDF.copy()
    if(conv_in == '10'):
        print("Converting to decimal...")
        for column in CAM_DF_conv.columns[colStart:colEnd]:
                for (idx, val) in CAM_DF_conv[column].dropna().items():
                    CAM_DF_conv.at[idx, column] = int(str(val), 16)
        print("Decimal conversion complete!")
    elif(conv_in == '2'):
        print("Converting to binary...")
        for column in CAM_DF_conv.columns[colStart:colEnd]:
            for (idx, val) in CAM_DF_conv[column].dropna().items():
                CAM_DF_conv.at[idx, column] = bin(int(str(val), 16))
        print("Binary conversion complete!")
    elif(conv_in == 'ascii'):
        print("Converting to ASCII...")
        for column in CAM_DF_conv.columns[colStart:colEnd]:
            for (idx, val) in CAM_DF_conv[column].dropna().items():
                decVal = int(str(val), 16)
                if(decVal > 127 or decVal < 33):
                    continue
                CAM_DF_conv.at[idx, column] = bytes.fromhex(str(val)).decode('ascii')
        print("ASCII conversion complete!")

    return CAM_DF_conv.iloc[:,:]

In [6]:
print(convert_num_base(5,9, 10))

Converting to decimal...
Decimal conversion complete!
        ID Specifier Index1 Index2 Sub-Index Data1 Data2 Data3 Data4
0      683       712      0      0         0     2     0     0   NaN
1      615       840     70      2        2D     0     0     0     0
2      682        71      0      0         0     0     0     0   NaN
3      687       826      0      0         0   132     3     5     0
4      618       840     60      7         1     0     0     0     0
...    ...       ...    ...    ...       ...   ...   ...   ...   ...
65527  215       NaN    NaN    NaN       NaN   NaN   NaN   NaN   NaN
65528  687       NaN    NaN    NaN       NaN   NaN   NaN   NaN   NaN
65529  687       NaN    NaN    NaN       NaN   NaN   NaN   NaN   NaN
65530  598       NaN    NaN    NaN       NaN   NaN   NaN   NaN   NaN
65531  598       NaN    NaN    NaN       NaN   NaN   NaN   NaN   NaN

[65532 rows x 9 columns]


In [52]:
def find_code_piece(read_point, target_code, column, targetDF = CAM_DF):

    if(read_point not in ('b', 'e')):
        print("Reading point input is invalid! \n'b' -> read from beginning\n'e' -> read from the end")
        return -1

    # if(not str(target_code).isnumeric()):
    #     print("Code input is not a number!")
    #     return -1

    if(column not in targetDF.columns):
        print("Column does not exist in target spreadsheet!")
        return -1

    CAM_DF_collec = pd.DataFrame()

    
    # Iterates every item in given column
    # Converts val in item and target code to string
        # slices value as the size of target code
        # (from beginning or end check reading point)
        # compares it to target code if the strings are the same
    # saves the whole line as a dataframe 
    # concats it to one data frame

    if(read_point == 'e'):
        for (idx, val) in targetDF[column].items():
            if(str(val)[-len(str(target_code)):] == str(target_code)):
                df = CAM_DF.iloc[[idx],:9]
                CAM_DF_collec = pd.concat([CAM_DF_collec, df])
    
    if(read_point == 'b'):
        for (idx, val) in targetDF[column].items():
            if(str(val)[:len(str(target_code))] == str(target_code)):
                df = CAM_DF.iloc[[idx],:9]
                CAM_DF_collec = pd.concat([CAM_DF_collec, df])
    

    return (CAM_DF_collec, target_code)


In [5]:
def save_as_excel(dataframe, file_name):
    dataframe.to_excel('../docs/code' + str(file_name) + '.xlsx')

In [16]:
save_as_excel(*find_code_piece('e', 60, "Specifier"))

In [21]:
data_stats_converted = convert_num_base(0, 1, 10)


Converting to decimal...
Decimal conversion complete!


In [23]:
save_as_excel(data_stats_converted.query('1408 <= ID <= 1535'), 'canid580h')
save_as_excel(data_stats_converted.query('1536 <= ID <= 1663'), 'canid600h')

In [36]:
save_as_excel(*find_code_piece('e', '40', "Specifier"))

In [37]:
save_as_excel(data_stats_converted.query('ID >= 1663'), 'greater67F')

In [39]:
CAM_DF.ID.unique()

array([683, 615, 682, 687, 618, 616, 597, 596, 315, 598, 595, 617, 215,
       686, 715, 717, 716, 718])

In [115]:
df_code_x = find_code_piece('e', '5', "ID", find_code_piece('e', '40', "Specifier")[0])[0]
df_code_y = find_code_piece('e', '5', "ID", find_code_piece('e', '43', "Specifier")[0])[0]

df_merged= pd.merge(df_code_x, df_code_y, how='inner', on=['Index1', 'Index2', 'Sub-Index']).drop_duplicates()
df_merged_sorted = convert_num_base(4, 5, '10', df_merged).sort_values(by=['Sub-Index'])
df_merged_sorted_ascii = convert_num_base(11, 15, 'ascii', df_merged_sorted)
save_as_excel(df_merged_sorted, 'ascii')

Converting to decimal...
Decimal conversion complete!
Converting to ASCII...
ASCII conversion complete!
